In [159]:
# importing functions
%run helper_functions.ipynb

### Prior transformation of data
Engineered features: filling missing values

-------

Standardizing
Log transformatio of targets and "cAlGa", "cAlIn", "cGaAl", "cInAl", "cInGa", "distAlO", "distGaAl", "distGaIn", "distGaO", "distInAl", "distInGa", "distInO", "spacegroup"

### Setting baseline

In [137]:
from lightgbm import LGBMRegressor
params_fe = {
    "boosting_type": "gbdt",
    "max_depth": 4,
    "learning_rate": 0.07,
    "n_estimators":90,
    "num_leaves": 15}
lgbm_fe = LGBMRegressor(**params_fe)
print(evaluate_CV(lgbm_fe, X_full, y_fe))
print(evaluate_CV(lgbm_fe, X_full, np.log1p(y_fe), metric=rmsle_scorer_exp))
params_be = {"boosting_type": "gbdt",
         "max_depth": 5,
         "learning_rate": 0.12,
         "n_estimators":100,
         "num_leaves": 20}
lgbm_be = LGBMRegressor(**params_be)
print(evaluate_CV(lgbm_be, X_full, y_be))
print(evaluate_CV(lgbm_be, X_full, np.log1p(y_be), metric=rmsle_scorer_exp))

0.031076171683833653
0.031166211129611043
0.08563496903988008
0.0864488272506217


In [104]:
lgbm_be.fit(X_full, np.log1p(y_be))
lgbm_fe.fit(X_full, np.log1p(y_fe))
save_results(np.expm1(lgbm_fe.predict(X_full_test)), np.expm1(lgbm_be.predict(X_full_test)), "baseline")

baseline kaggle private score: 0.06645

### Using PCA

In [62]:
X_full.columns

Index(['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree', 'Vatom', 'cAlGa', 'cAlIn', 'cAlO',
       'cGaAl', 'cGaIn', 'cGaO', 'cInAl', 'cInGa', 'cInO', 'distAlGa',
       'distAlIn', 'distAlO', 'distGaAl', 'distGaIn', 'distGaO', 'distInAl',
       'distInGa', 'distInO', 'elInt', 'qAl', 'qGa', 'qIn', 'qO', 'xEq',
       'centroSym', 'enantioMorph'],
      dtype='object')

In [160]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
cols =['lattice_vector_1_ang', 'lattice_vector_2_ang', 'lattice_vector_3_ang']
pca.fit(X_full[cols])
X_full["pca_lattice_vector"] = pca.transform(X_full[cols])
pca.fit(X_full_test[cols])
X_full_test["pca_lattice_vector"] = pca.transform(X_full_test[cols])

In [139]:
pca = PCA(n_components=1)
cols=['lattice_angle_alpha_degree', 'lattice_angle_beta_degree', 'lattice_angle_gamma_degree']
pca.fit(X_full[cols])
X_full["pca_lattice_angle"] = pca.transform(X_full[cols])
X_full_test["pca_lattice_angle"] = pca.transform(X_full_test[cols])

In [140]:
pca = PCA(n_components=1)
cols=['distAlGa','distAlIn', 'distAlO', 'distGaAl', 'distGaIn', 'distGaO', 'distInAl','distInGa', 'distInO']
pca.fit(X_full[cols])
X_full["pca_c_el"] = pca.transform(X_full[cols])
X_full_test["pca_c_el"] = pca.transform(X_full_test[cols])

In [66]:
from lightgbm import LGBMRegressor
params_fe = {
    "boosting_type": "gbdt",
    "max_depth": 4,
    "learning_rate": 0.07,
    "n_estimators":90,
    "num_leaves": 15}
lgbm_fe = LGBMRegressor(**params_fe)
print(evaluate_CV(lgbm_fe, X_full, y_fe))
print(evaluate_CV(lgbm_fe, X_full, np.log1p(y_fe), metric=rmsle_scorer_exp))

0.030955767184725546
0.030917546333049016


In [67]:
params_be = {"boosting_type": "gbdt",
         "max_depth": 5,
         "learning_rate": 0.12,
         "n_estimators":100,
         "num_leaves": 20}
lgbm_be = LGBMRegressor(**params_be)
print(evaluate_CV(lgbm_be, X_full, y_be))
print(evaluate_CV(lgbm_be, X_full, np.log1p(y_be), metric=rmsle_scorer_exp))

0.08687486302070033
0.08679369774557788


### Dropping columns

In [72]:
lgbm_be = LGBMRegressor(**params_be)
baseline = evaluate_CV(lgbm_be, X_full, y_be) 
print("baseline:", baseline)
for col in X_full.columns:
    new_score = evaluate_CV(lgbm_be, X_full.drop(col,axis=1), y_be)
    if baseline > new_score:
        print(col, new_score)

baseline: 0.08687486302070033
number_of_total_atoms 0.08678138756006411
percent_atom_al 0.08626871580109971
percent_atom_ga 0.08624452267794705
percent_atom_in 0.08604675641451362
lattice_vector_1_ang 0.0859571430189393
lattice_vector_2_ang 0.08630052865338995
lattice_angle_alpha_degree 0.08636961692467712
lattice_angle_beta_degree 0.08625627930406708
lattice_angle_gamma_degree 0.08684681269948502
cAlIn 0.08639971555908285
cAlO 0.08649704170187014
cGaO 0.086041197585078
cInAl 0.086337739066678
cInGa 0.08642664526328717
cInO 0.08640133617686747
distGaIn 0.08671536845591046
distInO 0.08676538307987212
elInt 0.08654957842358152
qGa 0.08624302909271027
qIn 0.08667318278357652
qO 0.08671432606794018
xEq 0.08679784494814262
pca_lattice_vector 0.08567267215052574
pca_lattice_angle 0.08653455610621351
pca_c_el 0.08623394530855275


In [100]:
lgbm_be = LGBMRegressor(**params_be)
cols_to_drop = ["pca_lattice_vector", "cAlIn"]
print(evaluate_CV(lgbm_be, X_full.drop(cols_to_drop, axis=1), y_be))
print(evaluate_CV(lgbm_be, X_full.drop(cols_to_drop, axis=1), np.log1p(y_be), metric=rmsle_scorer_exp))

0.08557809649469994
0.08636191504419785


In [82]:
lgbm_fe = LGBMRegressor(**params_fe)
baseline = evaluate_CV(lgbm_fe, X_full, y_fe) 
print("baseline:", baseline)
for col in X_full.columns:
    new_score = evaluate_CV(lgbm_fe, X_full.drop(col,axis=1), y_fe)
    if baseline > new_score:
        print(col, new_score)

baseline: 0.030955767184725546
percent_atom_al 0.03092520108531252
percent_atom_ga 0.030939667877196392
lattice_vector_1_ang 0.030935401187939016
lattice_angle_beta_degree 0.030949405927485286
Vatom 0.0308036268674025
cAlGa 0.03092153666837886
cAlIn 0.030723810323876923
cInAl 0.030828545865309302
distAlIn 0.030902373769311615
distGaO 0.030925790276504266
distInAl 0.030936477541494478
distInGa 0.03092631839701839
distInO 0.030932417792221722
qO 0.030926493602961215
centroSym 0.030934273091278986
pca_c_el 0.03093127052355891


In [189]:
lgbm_fe = LGBMRegressor(**params_fe)
cols_to_drop = ["cAlIn", "cInAl"]
print(evaluate_CV(lgbm_fe, X_full.drop(cols_to_drop, axis=1), y_fe))
print(evaluate_CV(lgbm_fe, X_full.drop(cols_to_drop, axis=1), np.log1p(y_fe), metric=rmsle_scorer_exp))

0.030623077027744437
0.030427157788177728


In [161]:
X_full_be = X_full.drop(["cAlIn"], axis=1)
X_full_fe = X_full.drop(["cAlIn", "cInAl"], axis=1)
X_full_test_be = X_full_test.drop(["cAlIn"], axis=1)
X_full_test_fe = X_full_test.drop(["cAlIn", "cInAl"], axis=1)

In [162]:
lgbm_be.fit(X_full_be, np.log1p(y_be))
lgbm_fe.fit(X_full_fe, np.log1p(y_fe))
save_results(np.expm1(lgbm_fe.predict(X_full_test_fe)), np.expm1(lgbm_be.predict(X_full_test_be)), "removal + pca")

Kaggle private score without pca: 0.06631

Kaggle private score with pca: 0.06681

In [1]:
# reset features
%run helper_functions.ipynb
X_full_be = X_full.drop(["cAlIn"], axis=1)
X_full_fe = X_full.drop(["cAlIn", "cInAl"], axis=1)
X_full_test_be = X_full_test.drop(["cAlIn"], axis=1)
X_full_test_fe = X_full_test.drop(["cAlIn", "cInAl"], axis=1)

## Parameter tuning

In [2]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.decomposition import PCA

In [3]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def param_search(model, grid, fe=True, transform_target=True):
    gs = GridSearchCV(model, grid, n_jobs=-1, scoring=(rmsle_scorer_exp if transform_target else rmsle_scorer), cv=3)
    target = (y_fe if fe else y_be)
    gs.fit(X_full_fe if fe else X_full_be , np.log1p(target) if transform_target else target)
    print(f"best score: {gs.best_score_}")
    print(f"best params: {gs.best_params_}")
    return gs.best_params_
    

In [95]:

model = Ridge()
grid = {
    "alpha":[0.0001,0.005,0.01,0.05,0.1,0.5,1,1.5,2,2.5],
    "random_state": [3]
}

ridge_params_be = param_search(model, grid, fe=False, transform_target=True)

ridge_params_fe = param_search(model, grid, fe=True, transform_target=True)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


best score: -0.09422746032504561
best params: {'alpha': 0.0001, 'random_state': 3}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


best score: -0.048699034134887395
best params: {'alpha': 2, 'random_state': 3}


In [103]:
model = ExtraTreesRegressor()
grid = {
    "n_estimators":[100,150,200,250],
    "max_depth":[None,10,12,15],
    "bootstrap":[True, False],
    "random_state": [3]
}

et_params_be = param_search(model, grid, fe=False, transform_target=True)
et_params_fe = param_search(model, grid, fe=True, transform_target=True)

best score: -0.08842670471149945
best params: {'bootstrap': True, 'max_depth': 12, 'n_estimators': 200, 'random_state': 3}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


best score: -0.02924787537074349
best params: {'bootstrap': True, 'max_depth': None, 'n_estimators': 250, 'random_state': 3}


In [108]:
et_params_fe={'bootstrap': True, 'max_depth': None, 'n_estimators': 250, 'random_state': 3}
et_fe = ExtraTreesRegressor(**et_params_fe)
params_be = {"boosting_type": "gbdt","max_depth": 5,"learning_rate": 0.12,"n_estimators":100,"num_leaves": 20}
lgbm_be = LGBMRegressor(**params_be)

lgbm_be.fit(X_full_be, np.log1p(y_be))
et_fe.fit(X_full_fe, np.log1p(y_fe))

save_results(np.expm1(et_fe.predict(X_full_test_fe)), np.expm1(lgbm_be.predict(X_full_test_be)), "et_fe + lgbm_be")

## Training 2-level stacking model

In [10]:
from sklearn.model_selection import train_test_split
X_train_fe, X_val_fe, y_train_fe, y_val_fe = train_test_split(X_full_fe, y_fe, train_size=0.2, random_state=1)
X_train_be, X_val_be, y_train_be, y_val_be = train_test_split(X_full_be, y_be, train_size=0.2, random_state=1)

In [17]:
params_be = {"boosting_type": "gbdt",
         "max_depth": 5,
         "learning_rate": 0.12,
         "n_estimators":100,
         "num_leaves": 20}
lgbm_be = LGBMRegressor(**params_be)

params_fe = {
    "boosting_type": "gbdt",
    "max_depth": 4,
    "learning_rate": 0.07,
    "n_estimators":90,
    "num_leaves": 15}
lgbm_fe = LGBMRegressor(**params_fe)

lgbm_be.fit(X_train_be, np.log1p(y_train_be))
lgbm_fe.fit(X_train_fe, np.log1p(y_train_fe))
print("fe rmsle:",rmsle(y_val_fe, np.expm1(lgbm_fe.predict(X_val_fe.values))))
print("be rmsle:",rmsle(y_val_be, np.expm1(lgbm_be.predict(X_val_be.values))))

fe rmsle: 0.03395452748503327
be rmsle: 0.09719871575692053


### Preparing individual classifiers

#### Formation energy classifiers

In [5]:
rf_fe = RandomForestRegressor(
    criterion='mse', 
    n_estimators=1000, 
    max_depth=35, 
    min_samples_split=4, 
    random_state=1)

mlp_fe = MLPRegressor(
    activation="relu", 
    solver="lbfgs", 
    learning_rate="invscaling", 
    hidden_layer_sizes=(50, 50))

knn_fe = KNeighborsRegressor(
    n_neighbors=5, 
    weights='uniform', 
    metric='manhattan')

params_fe = {
    "boosting_type": "gbdt",
    "max_depth": 4,
    "learning_rate": 0.07,
    "n_estimators":60,
    "num_leaves": 15}

et_params_fe={'bootstrap': True, 
              'max_depth': None, 
              'n_estimators': 250, 
              'random_state': 3}
et_fe = ExtraTreesRegressor(**et_params_fe)


params_be = {"boosting_type": "gbdt","max_depth": 5,"learning_rate": 0.12,"n_estimators":100,"num_leaves": 20}
lgbm_fe = LGBMRegressor(**params_fe)

In [29]:
clf_fe =  [rf_fe, mlp_fe, knn_fe, lgbm_fe, et_fe]

In [12]:
meta_fe = Ridge()

In [13]:
stack_fe = StackingCVRegressor(regressors=clf_fe,meta_regressor=meta_fe, n_jobs=-1, random_state=3, cv=3)

In [23]:
stack_fe.fit(X_train_fe.values, y_train_fe.values)
print(rmsle(y_val_fe, stack_fe.predict(X_val_fe.values)))
#print(evaluate_CV(stack_fe, X_full_fe.values, np.log1p(y_fe.values), metric=rmsle_scorer_exp))

0.03380932097810664


In [27]:
stack_fe.fit(X_full_fe.values, y_fe.values)
params_be = {"boosting_type": "gbdt","max_depth": 5,"learning_rate": 0.12,"n_estimators":100,"num_leaves": 20}
lgbm_be = LGBMRegressor(**params_be)

lgbm_be.fit(X_full_be, np.log1p(y_be))
save_results(stack_fe.predict(X_full_test_fe), np.expm1(lgbm_be.predict(X_full_test_be)), "stacking fe")

#### Bandgap energy classifiers

In [174]:
mlp_be = MLPRegressor(
    activation="tanh", 
    solver="lbfgs", 
    learning_rate="invscaling", 
    hidden_layer_sizes=(50, 100, 50))

svr_be = SVR(kernel="poly", gamma="scale")

params_be = {"boosting_type": "gbdt",
         "max_depth": 5,
         "learning_rate": 0.12,
         "n_estimators":100,
         "num_leaves": 20}

lgbm_be = LGBMRegressor(**params_be)


In [207]:
meta_be = Ridge()

In [208]:
clf_be = [mlp_be, svr_be, lgbm_be]

In [209]:
stack_be = StackingCVRegressor(regressors=clf_be, meta_regressor=meta_be, random_state=3, n_jobs=-1)



In [28]:
stack_be.fit(X_train_be.values, y_train_be.values)
print(rmsle(y_val_be, stack_be.predict(X_val_be.values)))
print(evaluate_CV(stack_be, X_full_be.values, np.log1p(y_be.values), metric=rmsle_scorer_exp))

NameError: name 'stack_be' is not defined